In [1]:
import uuid
from kubeflow import fairing
from kubeflow.fairing.kubernetes import utils as k8s_utils

# docker hub username
CONTAINER_REGISTRY = 'seokii'

# namespace
namespace = 'kubeflow-user-example-com'

# job name 지정
job_name = f'sklearn-rf-iris-{uuid.uuid4().hex[:4]}'

# command 정의
command=["python", "sklearn_rf_iris.py", "--model_path", "/mnt/pv/models/sklearn/iris"]

# output_map 정의
output_map = {
    "Dockerfile": "Dockerfile",
    "sklearn_rf_iris.py": "sklearn_rf_iris.py"
}

# fairing 전처리기
fairing.config.set_preprocessor('python', command=command, path_prefix="/", output_map=output_map)

# cluster builder ContextSource를 minio로 정의
s3_endpoint = 'minio-service.kubeflow.svc.cluster.local:9000'
minio_endpoint = "http://"+s3_endpoint
minio_username = "minio"
minio_key = "minio123"
minio_region = "us-east-1"

# minio_context사용
from kubeflow.fairing.builders.cluster.minio_context import MinioContextSource
minio_context_source = MinioContextSource(endpoint_url=minio_endpoint, minio_secret=minio_username, minio_secret_key=minio_key, region_name=minio_region)

# fairing 빌더
fairing.config.set_builder('cluster', registry=CONTAINER_REGISTRY, image_name="sklearn_rf_iris", dockerfile_path="Dockerfile",
                          context_source=minio_context_source)

# fairing 배포
fairing.config.set_deployer('job', namespace=namespace, job_name=job_name,
                            pod_spec_mutators=[k8s_utils.mounting_pvc(pvc_name='kserve-models-pvc', pvc_mount_path='/mnt/pv')],
                            cleanup=False, stream_log=True)
fairing.config.run()

[W 230110 18:19:15 utils:50] The function mounting_pvc has been deprecated,                     please use `volume_mounts`
[I 230110 18:19:15 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.base.BasePreProcessor object at 0x7fa9c823b2e0>
[I 230110 18:19:15 config:136] Using builder: <kubeflow.fairing.builders.cluster.cluster.ClusterBuilder object at 0x7fa938bf75b0>
[I 230110 18:19:15 config:138] Using deployer: <kubeflow.fairing.deployers.job.job.Job object at 0x7fa93cfc5e20>
[I 230110 18:19:15 cluster:46] Building image using cluster builder.
[I 230110 18:19:15 base:107] Creating docker context: /tmp/fairing_context_ivpdkdch
[W 230110 18:19:17 manager:298] Waiting for fairing-builder-rk2kp-4wfk9 to start...
[W 230110 18:19:17 manager:298] Waiting for fairing-builder-rk2kp-4wfk9 to start...
[W 230110 18:19:17 manager:298] Waiting for fairing-builder-rk2kp-4wfk9 to start...
[I 230110 18:19:18 manager:303] Pod started running True


INFO[0001] Retrieving image manifest python:3.8.5
INFO[0003] Retrieving image manifest python:3.8.5
INFO[0005] Built cross stage deps: map[]
INFO[0005] Retrieving image manifest python:3.8.5
INFO[0006] Retrieving image manifest python:3.8.5
INFO[0008] Executing 0 build triggers
INFO[0008] Unpacking rootfs as cmd RUN pip install scikit-learn==1.2.0 joblib requires it.
INFO[0040] Taking snapshot of full filesystem...
INFO[0041] Resolving 31140 paths
INFO[0042] RUN pip install scikit-learn==1.2.0 joblib
INFO[0042] cmd: /bin/sh
INFO[0042] args: [-c pip install scikit-learn==1.2.0 joblib]
INFO[0042] Running: [/bin/sh -c pip install scikit-learn==1.2.0 joblib]
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
INFO[0053] Using files from context: [/kaniko/buildcontext/sklearn_rf_iris.py]
INFO[0053] ADD sklearn_rf_iris.py /
INFO[0053] Taking snapshot of full filesystem...
INFO[0053] Resolving 37059 paths


[W 230110 18:20:40 job:101] The job sklearn-rf-iris-d1b3 launched.
[W 230110 18:20:42 manager:298] Waiting for sklearn-rf-iris-d1b3-trlf7 to start...
[W 230110 18:20:46 manager:298] Waiting for sklearn-rf-iris-d1b3-trlf7 to start...
[W 230110 18:20:46 manager:298] Waiting for sklearn-rf-iris-d1b3-trlf7 to start...
[I 230110 18:21:37 manager:303] Pod started running True


(<kubeflow.fairing.preprocessors.base.BasePreProcessor at 0x7fa9c823b2e0>,
 <kubeflow.fairing.deployers.job.job.Job at 0x7fa93cfc5e20>)